In [ ]:
import sys,os
dev_mode = True
preset = 'python'

if dev_mode:
    try:
        install_prefix = os.environ['CMAKE_INSTALL_PREFIX']
        print(f'Using CMAKE_INSTALL_PREFIX={install_prefix}')
    except KeyError:
        install_prefix = f'../../build/{preset}/install'
    print(f'Using install_prefix={install_prefix}')
    if os.path.exists(f'{install_prefix}/lib/python3/dist-packages'):
        if os.path.abspath(install_prefix)=='/usr':
            sys.path.append(os.path.abspath(f'{install_prefix}/lib/python3/dist-packages'))
        else:
            sys.path.insert(0, os.path.abspath(f'{install_prefix}/lib/python3/dist-packages'))
    print(sys.path)

In [ ]:
import feelpp
from feelpp.meshmover import *
from feelpp.integrate import integrate
from feelpp.measure import measure
import feelpp.toolboxes.core as core
from feelpp.toolboxes.cfpdes import *
import feelpp.quality as quality
import numpy as np
print(f"feelpp loaded from {feelpp.__file__}")

In [ ]:
os.getcwd()

In [ ]:
sys.argv = ['notebook-mesh']
e = feelpp.Environment(sys.argv,config=feelpp.localRepository(sys.argv[0]),opts = core.toolboxes_options("coefficient-form-pdes", "cfpdes"))

In [ ]:
os.getcwd()

In [ ]:
geo = {
    '2': feelpp.create_rectangle(),
    '3': feelpp.create_box(),
}
mesh_name = geo['2'][0]
print(mesh_name)


In [ ]:
laplacian={
    "Name": "Laplacian",
    "ShortName": "Laplacian",
    "Models":
    {
        "cfpdes":{ "equations":"laplace" },
        "laplace":{
            "setup":{
                "unknown":{
                    "basis":"Pchv1",
                    "name":"u",
                    "symbol":"u"
                },
                "coefficients":{ "c":"1", "f":"{1,1}"}
            }
        }
    },
    "Materials": { "Omega": { "markers":"Omega" } },
    "BoundaryConditions":
    {
        "laplace":
        {
            "Dirichlet":
            {
                "GammaMove":
                {
                    "markers":["Gamma_1"],
                    "expr":"{0,amplitude*x*(1-x)+0.1}:x:amplitude"
                },
                "zero":
                {
                    "markers":["Gamma_2","Gamma_3","Gamma_4"],
                    "expr":"{0,0}"
                }
            }
        }
    },
    "PostProcess":
    {
        "cfpdes":
        {
            "Exports":
            {
                "fields":["all"]
            }
        }
    }
}


In [ ]:
mesh = feelpp.load(feelpp.mesh(dim=2), geo['2'][0], 0.1)
print("Test of the mesh quality")
hclose = 0.01
hfar = 0.025
q = quality.etaQ(mesh).min()
print(f"mesh quality criterion={q}\n")
if q < 0.4 :
    print("Remeshing\n")
    mesh, cpt = feelpp.remesh(mesh=mesh, metric=f"gradedls({hclose},{hfar})",required_elts=[],required_facets=["Gamma_2","Gamma_3","Gamma_4"],parent=None)

for iter in range(0,2):
    toolbox=cfpdes(dim=2)
    toolbox.setMesh(mesh)
    toolbox.setModelProperties(laplacian)
    toolbox.addParameterInModelProperties("amplitude",0.05)

    #
    toolbox.init(buildModelAlgebraicFactory=True)
    toolbox.solve()
    toolbox.exportResults()

    u=toolbox.pde("laplace").fieldUnknown()
    #assert(abs(M-2)<1e-10)
    M=measure(range=feelpp.elements(mesh))
    print(f"[iter {iter}] before : {M}")
    mesh=meshMove(mesh,u)
    M=measure(range=feelpp.elements(mesh))
    print(f"[iter {iter}] after : {M}")
    q = quality.etaQ(mesh).min()
    print(f"mesh quality criterion={q}\n")
    if q < 0.4 :
        print("Remeshing\n")
        mesh, cpt = feelpp.remesh(mesh=mesh, metric=f"gradedls({hclose},{hfar})",required_elts=[],required_facets=[],parent=None)